In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
eval_test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.Grayscale(num_output_channels=3),  # convert 1-channel → 3-channel
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [4]:
from torch.utils.data import random_split

train_dataset = datasets.ImageFolder(r'C:\Radiologist_helper\datasets\brain_mri\Training', transform=train_transform)

# Split: 80% train, 20% val
val_size = int(0.2 * len(train_dataset))
train_size = len(train_dataset) - val_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)

test_dataset  = datasets.ImageFolder(r'C:\Radiologist_helper\datasets\brain_mri\Testing', transform=eval_test_transform)
test_loader   = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [5]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
class BrainMRIResNet50(nn.Module):
    def __init__(self, num_classes=4, dropout_rate=0.5):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')

        # Freeze all layers
        for param in self.model.parameters():
            param.requires_grad = False

        # Optional: unfreeze last block (layer4)
        for param in self.model.layer4.parameters():
            param.requires_grad = True

        # Replace fully connected layer
        in_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [7]:
def train_model(model, criterion, optimizer, epochs=10, patience=5):
    start = time.time()
    best_acc = 0.0
    best_loss = float('inf')
    patience_counter = 0

    all_labels, all_predictions = [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for batch_num, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

            if (batch_num + 1) % 10 == 0:
                print(f"Batch: {batch_num+1}, Epoch: {epoch+1}, Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{epochs}] - Avg Train Loss: {epoch_loss:.4f}")

        # ---------- Validation ----------
        model.eval()
        correct, total = 0, 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        val_acc = 100 * correct / total
        print(f"Validation Loss: {val_loss:.4f} | Accuracy: {val_acc:.2f}%")

        # --- Check for improvement ---
        if val_loss < best_loss or val_acc > best_acc:
            best_loss = val_loss
            best_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), r'C:\Radiologist_helper\trained_models\brain_mri_model.pth')
        else:
            patience_counter += 1
            print(f"No improvement. Patience: {patience_counter}/{patience}")
            if patience_counter >= patience:
                break

    end = time.time()
    print(f"\nTraining complete in {(end - start):.2f}s")
    print(f"Best Validation Accuracy: {best_acc:.2f}%")

    return all_labels, all_predictions


In [8]:
model = BrainMRIResNet50(num_classes=4, dropout_rate=0.5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)



In [9]:
labels, predictions = train_model(model, criterion, optimizer, epochs=10)

Batch: 10, Epoch: 1, Loss: 1.2964
Batch: 20, Epoch: 1, Loss: 0.9750
Batch: 30, Epoch: 1, Loss: 0.9076
Batch: 40, Epoch: 1, Loss: 0.5675
Batch: 50, Epoch: 1, Loss: 0.6022
Batch: 60, Epoch: 1, Loss: 0.5415
Batch: 70, Epoch: 1, Loss: 0.3455
Batch: 80, Epoch: 1, Loss: 0.2064
Batch: 90, Epoch: 1, Loss: 0.2885
Batch: 100, Epoch: 1, Loss: 0.2093
Batch: 110, Epoch: 1, Loss: 0.3202
Batch: 120, Epoch: 1, Loss: 0.3075
Batch: 130, Epoch: 1, Loss: 0.2877
Batch: 140, Epoch: 1, Loss: 0.3669
Epoch [1/10] - Avg Train Loss: 0.5158
Validation Loss: 0.1876 | Accuracy: 93.08%
Batch: 10, Epoch: 2, Loss: 0.2299
Batch: 20, Epoch: 2, Loss: 0.4680
Batch: 30, Epoch: 2, Loss: 0.3734
Batch: 40, Epoch: 2, Loss: 0.1602
Batch: 50, Epoch: 2, Loss: 0.3237
Batch: 60, Epoch: 2, Loss: 0.1380
Batch: 70, Epoch: 2, Loss: 0.0820
Batch: 80, Epoch: 2, Loss: 0.2100
Batch: 90, Epoch: 2, Loss: 0.2309
Batch: 100, Epoch: 2, Loss: 0.2740
Batch: 110, Epoch: 2, Loss: 0.1788
Batch: 120, Epoch: 2, Loss: 0.2633
Batch: 130, Epoch: 2, Loss:

In [10]:
from sklearn.metrics import classification_report

report = classification_report(labels, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.95      0.96      2730
           1       0.93      0.94      0.94      2670
           2       0.98      0.99      0.99      2990
           3       0.98      0.98      0.98      3030

    accuracy                           0.97     11420
   macro avg       0.97      0.97      0.97     11420
weighted avg       0.97      0.97      0.97     11420



In [13]:
def evaluate_test(model, test_loader, model_path=r'C:\Radiologist_helper\trained_models\brain_mri_model.pth'):
    device = next(model.parameters()).device
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(images)
            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).int()

            total += labels.size(0)
            correct += (preds == labels.int()).sum().item()

    test_acc = 100 * correct / total
    print(f"*** Test Accuracy: {test_acc:.2f}% ***")
    return test_acc

In [14]:
test_acc = evaluate_test(model, test_loader)
print(f"*** Test Accuracy: {test_acc:.2f}% ***")

*** Test Accuracy: 90.31% ***
*** Test Accuracy: 90.31% ***
